In [3]:
import pandas as pd
import numpy as np
df3 = pd.read_csv("/Users/rukhshan/Documents/GitHub/adv_ml/GenderProject/data/Pakistan/pak_extracted_combined.csv")
df3 = df3[['link', 'datetime', 'title', 'text', 'authors', 'summary', 'keywords']]
df3 = df3[['link', 'datetime', 'title', 'text', 'authors', 'summary', 'keywords']]

df3['newspaper'] = ""
df3['newspaper'] = np.where(df3.link.str.contains("nation.com"), "nation", df3.newspaper)
df3['newspaper'] = np.where(df3.link.str.contains("news.com"), "news", df3.newspaper)
df3['newspaper'] = np.where(df3.link.str.contains("dawn.com"), "dawn", df3.newspaper)

In [ ]:
nation_df = df3[df3['newspaper']=='nation']
dawn_df = df3[df3['newspaper']=='dawn']
thenews_df = df3[df3['newspaper']=='news']


In [11]:
news_dates_df = pd.read_csv("/Users/rukhshan/Documents/GitHub/final-project-rukhshanarifm/news_datetime.csv")
news_final_df = news_dates_df.merge(thenews_df, on='link')
dawn_df['datetime'] = pd.to_datetime(dawn_df['datetime'])
nation_2020ow = nation_df[(nation_df['datetime'].str.contains("2020")) | (nation_df['datetime'].str.contains("2021"))]
dawn_2020ow = dawn_df[(dawn_df['datetime'].dt.year == 2020) | (dawn_df['datetime'].dt.year == 2021)]
news_2020ow = news_2020ow = news_final_df[(news_final_df['datetime_x'].str.contains("2020")) | (news_final_df['datetime_x'].str.contains("2021"))]

/Users/rukhshan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
import sys
sys.path.append(r"/Users/rukhshan/Documents/GitHub/adv_ml/GenderProject")

import pandas as pd 
import numpy as np 
import gensim 
import spacy 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
## NN based spanish sentiment scorer built on Tensorflow + Keras
from pre_process.text_utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The News

In [29]:
news_2020ow["title"] = news_2020ow["title"].str.pad(1, side ='right', fillchar =' ')
news_2020ow["complete_text"]  = news_2020ow.title.str.cat(news_2020ow.text).astype(str)

/Users/rukhshan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/rukhshan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [54]:
# Normalize and tokenize all documents

'''
nlp_english = spacy.load("en_core_web_sm")

news_2020ow['tokenized_sentences'] = news_2020ow['complete_text'].apply(lambda x: 
                                                                      [word_tokenize(s, model=nlp_english)
                                                                      for s in sent_tokenize(x)])
news_2020ow['normalized_sentences'] = news_2020ow['tokenized_sentences'].apply(lambda x:
                                                                            [normalize_tokens(s, model=nlp_english)
                                                                            for s in x])
'''

'\nnlp_english = spacy.load("en_core_web_sm")\n\nnews_2020ow[\'tokenized_sentences\'] = news_2020ow[\'complete_text\'].apply(lambda x: \n                                                                      [word_tokenize(s, model=nlp_english)\n                                                                      for s in sent_tokenize(x)])\nnews_2020ow[\'normalized_sentences\'] = news_2020ow[\'tokenized_sentences\'].apply(lambda x:\n                                                                            [normalize_tokens(s, model=nlp_english)\n                                                                            for s in x])\n'

In [35]:
news_w2v  = gensim.models.word2vec.Word2Vec(news_2020ow['normalized_sentences'].sum(),window=10)

In [46]:
news_w2v.wv.most_similar("women")

[('gender', 0.9991583228111267),
 ('based', 0.9987853169441223),
 ('domestic', 0.9964468479156494),
 ('empowerment', 0.9953795075416565),
 ('rights', 0.9945817589759827),
 ('violence', 0.9940029978752136),
 ('forms', 0.9933726191520691),
 ('country', 0.992571234703064),
 ('child', 0.9925147294998169),
 ('harassment', 0.9923556447029114)]

In [47]:
news_w2v.wv.most_similar("girl")

[('boy', 0.9984163641929626),
 ('man', 0.9973467588424683),
 ('minor', 0.9961004257202148),
 ('allegedly', 0.9958158135414124),
 ('kidnapped', 0.99568772315979),
 ('raping', 0.995495080947876),
 ('limits', 0.9948391318321228),
 ('arrested', 0.9945178627967834),
 ('abducted', 0.9944660067558289),
 ('teenage', 0.994373083114624)]

## Dawn

In [55]:
'''
dawn_2020ow["title"] = dawn_2020ow["title"].str.pad(1, side ='right', fillchar =' ')
dawn_2020ow["complete_text"]  = dawn_2020ow.title.str.cat(dawn_2020ow.text).astype(str)
dawn_2020ow['tokenized_sentences'] = dawn_2020ow['complete_text'].apply(lambda x: 
                                                                      [word_tokenize(s, model=nlp_english)
                                                                      for s in sent_tokenize(x)])
dawn_2020ow['normalized_sentences'] = dawn_2020ow['tokenized_sentences'].apply(lambda x:
                                                                            [normalize_tokens(s, model=nlp_english)
                                                                            for s in x])
dawn_w2v  = gensim.models.word2vec.Word2Vec(dawn_2020ow['normalized_sentences'].sum(),window=10)
'''

'\ndawn_2020ow["title"] = dawn_2020ow["title"].str.pad(1, side =\'right\', fillchar =\' \')\ndawn_2020ow["complete_text"]  = dawn_2020ow.title.str.cat(dawn_2020ow.text).astype(str)\ndawn_2020ow[\'tokenized_sentences\'] = dawn_2020ow[\'complete_text\'].apply(lambda x: \n                                                                      [word_tokenize(s, model=nlp_english)\n                                                                      for s in sent_tokenize(x)])\ndawn_2020ow[\'normalized_sentences\'] = dawn_2020ow[\'tokenized_sentences\'].apply(lambda x:\n                                                                            [normalize_tokens(s, model=nlp_english)\n                                                                            for s in x])\ndawn_w2v  = gensim.models.word2vec.Word2Vec(dawn_2020ow[\'normalized_sentences\'].sum(),window=10)\n'

In [50]:
dawn_w2v.wv.most_similar("women")

[('ali', 0.9998335242271423),
 ('said', 0.9998325705528259),
 ('violence', 0.9998313784599304),
 ('state', 0.9998263120651245),
 ('children', 0.9998194575309753),
 ('law', 0.9998192191123962),
 ('minister', 0.99981689453125),
 ('country', 0.9998145699501038),
 ('justice', 0.9998140335083008),
 ('police', 0.9998077154159546)]

In [51]:
dawn_w2v.wv.most_similar("girl")

[('said', 0.9998308420181274),
 ('police', 0.9997838139533997),
 ('ali', 0.9997808337211609),
 ('family', 0.9997771382331848),
 ('sindh', 0.9997691512107849),
 ('pakistan', 0.9997639060020447),
 ('government', 0.9997497200965881),
 ('karachi', 0.9997479319572449),
 ('suspect', 0.9997455477714539),
 ('justice', 0.9997419118881226)]

## Nation

In [56]:
'''
nation_2020ow["title"] = nation_2020ow["title"].str.pad(1, side ='right', fillchar =' ')
nation_2020ow["complete_text"]  = nation_2020ow.title.str.cat(nation_2020ow.text).astype(str)
nation_2020ow['tokenized_sentences'] = nation_2020ow['complete_text'].apply(lambda x: 
                                                                      [word_tokenize(s, model=nlp_english)
                                                                      for s in sent_tokenize(x)])
nation_2020ow['normalized_sentences'] = nation_2020ow['tokenized_sentences'].apply(lambda x:
                                                                            [normalize_tokens(s, model=nlp_english)
                                                                            for s in x])
nation_w2v  = gensim.models.word2vec.Word2Vec(nation_2020ow['normalized_sentences'].sum(),window=10)
'''

'\nnation_2020ow["title"] = nation_2020ow["title"].str.pad(1, side =\'right\', fillchar =\' \')\nnation_2020ow["complete_text"]  = nation_2020ow.title.str.cat(nation_2020ow.text).astype(str)\nnation_2020ow[\'tokenized_sentences\'] = nation_2020ow[\'complete_text\'].apply(lambda x: \n                                                                      [word_tokenize(s, model=nlp_english)\n                                                                      for s in sent_tokenize(x)])\nnation_2020ow[\'normalized_sentences\'] = nation_2020ow[\'tokenized_sentences\'].apply(lambda x:\n                                                                            [normalize_tokens(s, model=nlp_english)\n                                                                            for s in x])\nnation_w2v  = gensim.models.word2vec.Word2Vec(nation_2020ow[\'normalized_sentences\'].sum(),window=10)\n'

In [52]:
nation_w2v.wv.most_similar("women")

[('pakistan', 0.9998694658279419),
 ('society', 0.9998290538787842),
 ('government', 0.9998155236244202),
 ('country', 0.9998108744621277),
 ('laws', 0.9998046159744263),
 ('social', 0.9998027682304382),
 ('children', 0.999799907207489),
 ('gender', 0.9997987151145935),
 ('victims', 0.9997978806495667),
 ('child', 0.9997929334640503)]

In [53]:
nation_w2v.wv.most_similar("girl")

[('boy', 0.9997513294219971),
 ('area', 0.99973464012146),
 ('killed', 0.999720573425293),
 ('man', 0.9997128248214722),
 ('police', 0.9997102618217468),
 ('allegedly', 0.9997049570083618),
 ('year', 0.9997023344039917),
 ('woman', 0.9996995329856873),
 ('lahore', 0.9996957778930664),
 ('raped', 0.999686598777771)]